# Terminal commands to preview compiled files

`zcat uniref90_mapping_debug.tsv.gz | head -n 20`

# Curent best code

In [10]:
# duckdb_debug_uniref.py
import duckdb
from pathlib import Path

# Point to your processed mapping files
base = Path("/scratch/pranamlab/sophie/interactome/interactome/data_files/processed")
u50_path = base / "uniref50/uniref50_mapping.tsv.gz"
u90_path = base / "uniref90/uniref90_mapping.tsv.gz"

# Use a DB file (full, not debug subset)
con = duckdb.connect("uniref.duckdb")

# (Re)create tables if needed — if you want to force refresh, DROP TABLE first.
con.execute("CREATE TABLE IF NOT EXISTS uniref50 AS "
            f"SELECT "
            f"  uniprot_acc    AS member_acc, "
            f"  uniprot_id, "
            f"  is_primary, "
            f"  uniref_cluster AS uniref50_id "
            f"FROM read_csv_auto('{u50_path}', header=True, delim='\t');")

con.execute("CREATE TABLE IF NOT EXISTS uniref90 AS "
            f"SELECT "
            f"  uniprot_acc    AS member_acc, "
            f"  uniprot_id, "
            f"  is_primary, "
            f"  uniref_cluster AS uniref90_id "
            f"FROM read_csv_auto('{u90_path}', header=True, delim='\t');")


# Optional indexes for faster lookups
con.execute("CREATE INDEX IF NOT EXISTS idx_u50_acc ON uniref50(member_acc);")
con.execute("CREATE INDEX IF NOT EXISTS idx_u90_acc ON uniref90(member_acc);")
con.execute("CREATE INDEX IF NOT EXISTS idx_u50_id  ON uniref50(uniref50_id);")
con.execute("CREATE INDEX IF NOT EXISTS idx_u90_id  ON uniref90(uniref90_id);")

def query_candidates(con, seed_acc: str, primary_only: bool = False):
    # optional filter: only keep primary accessions
    primary_filter = "AND c.is_primary = 1" if primary_only else ""

    con.execute(f"""
        WITH seed AS (
          SELECT u50.uniref50_id
          FROM uniref50 u50
          WHERE u50.member_acc = ?
          LIMIT 1
        ),
        same_u50 AS (
          SELECT u50.member_acc,
                 u50.uniref50_id,
                 u50.is_primary
          FROM uniref50 u50
          JOIN seed s
            ON u50.uniref50_id = s.uniref50_id
        ),
        same_u50_with_u90 AS (
          SELECT s.member_acc,
                 s.uniref50_id,
                 s.is_primary,
                 u90.uniref90_id
          FROM same_u50 s
          LEFT JOIN uniref90 u90
            ON s.member_acc = u90.member_acc
        ),
        seed_u90 AS (
          SELECT u90.uniref90_id
          FROM uniref90 u90
          WHERE u90.member_acc = ?
          LIMIT 1
        )
        SELECT c.member_acc
        FROM same_u50_with_u90 c, seed_u90
        WHERE c.member_acc <> ?
          AND (
                c.uniref90_id IS NULL
             OR c.uniref90_id <> seed_u90.uniref90_id
          )
          {primary_filter};
    """, [seed_acc, seed_acc, seed_acc])

    return [r[0] for r in con.fetchall()]


def debug_uniref_for_seed(con, seed_acc: str, limit: int = 20):
    print(f"\n================= DEBUG FOR SEED {seed_acc} =================")

    # 0) Show seed rows in uniref50 / uniref90
    print("\n[0] Seed rows in uniref50:")
    rows50 = con.execute(
        "SELECT * FROM uniref50 WHERE member_acc = ?;", [seed_acc]
    ).fetchall()
    print(rows50 if rows50 else "  (none)")

    print("\n[0] Seed rows in uniref90:")
    rows90 = con.execute(
        "SELECT * FROM uniref90 WHERE member_acc = ?;", [seed_acc]
    ).fetchall()
    print(rows90 if rows90 else "  (none)")

    if not rows50:
        print("\n!! Seed is not present in uniref50 at all. Everything else will be empty.")
        return

    # 1) Things in the same UniRef50 cluster
    print("\n[1] Same UniRef50 cluster as seed:")
    same_u50 = con.execute("""
        WITH seed AS (
          SELECT uniref50_id
          FROM uniref50
          WHERE member_acc = ?
          LIMIT 1
        )
        SELECT u50.member_acc, u50.uniref50_id
        FROM uniref50 u50
        JOIN seed s USING(uniref50_id);
    """, [seed_acc]).fetchall()
    print(f"  total: {len(same_u50)}")
    for row in same_u50[:limit]:
        print(" ", row)

    # 2) Things in the same UniRef90 cluster
    if not rows90:
        print("\n[2] Seed has no UniRef90 entry (not in uniref90). Skipping 'same_u90' listing.")
        same_u90 = []
    else:
        print("\n[2] Same UniRef90 cluster as seed:")
        same_u90 = con.execute("""
            WITH seed AS (
              SELECT uniref90_id
              FROM uniref90
              WHERE member_acc = ?
              LIMIT 1
            )
            SELECT u90.member_acc, u90.uniref90_id
            FROM uniref90 u90
            JOIN seed s USING(uniref90_id);
        """, [seed_acc]).fetchall()
        print(f"  total: {len(same_u90)}")
        for row in same_u90[:limit]:
            print(" ", row)

    # 3) Things in the same UniRef50 cluster but NOT in the same UniRef90 cluster
    print("\n[3] Same UniRef50 cluster, but NOT same UniRef90 cluster as seed:")
    diff_u50_not_u90 = con.execute("""
        WITH seed50 AS (
          SELECT uniref50_id
          FROM uniref50
          WHERE member_acc = ?
          LIMIT 1
        ),
        seed90 AS (
          SELECT uniref90_id
          FROM uniref90
          WHERE member_acc = ?
          LIMIT 1
        ),
        same50 AS (
          SELECT u50.member_acc, u50.uniref50_id
          FROM uniref50 u50
          JOIN seed50 s USING(uniref50_id)
        )
        SELECT s.member_acc, s.uniref50_id, u90.uniref90_id
        FROM same50 s
        LEFT JOIN uniref90 u90 USING(member_acc), seed90
        WHERE s.member_acc <> ?
          AND (
               u90.uniref90_id IS NULL
            OR u90.uniref90_id <> seed90.uniref90_id
          );
    """, [seed_acc, seed_acc, seed_acc]).fetchall()

    print(f"  total: {len(diff_u50_not_u90)}")
    for row in diff_u50_not_u90[:limit]:
        print(" ", row)

    print("\n[4] query_candidates() result (should match [3] but only member_acc):")
    candidates = query_candidates(con, seed_acc)
    print(f"  total: {len(candidates)}")
    print(" ", candidates[:limit])

In [12]:
# Example usage:
if __name__ == "__main__":
    test_seed = "Q8WZ42-11" #"Q12778"
    debug_uniref_for_seed(con, test_seed, limit=30)
    cands_primary = query_candidates(con, "Q8WZ42-11", primary_only=True)
    print(f"cands_primary: {cands_primary}")


================= DEBUG FOR SEED Q8WZ42-11 =================

[0] Seed rows in uniref50:
[('Q8WZ42-11', 'Q8WZ42-11', 1, 'UniRef50_Q8WZ42')]

[0] Seed rows in uniref90:
[('Q8WZ42-11', 'Q8WZ42-11', 1, 'UniRef90_Q8WZ42')]

[1] Same UniRef50 cluster as seed:
  total: 2277
  ('A0A852IEX3', 'UniRef50_Q8WZ42')
  ('A0A7K9EZM8', 'UniRef50_Q8WZ42')
  ('A0A852KVB2', 'UniRef50_Q8WZ42')
  ('A0A7K5ZDN2', 'UniRef50_Q8WZ42')
  ('A0A7L3BZI0', 'UniRef50_Q8WZ42')
  ('A0A7L3M7S7', 'UniRef50_Q8WZ42')
  ('A0A851UD84', 'UniRef50_Q8WZ42')
  ('A0A7K8NQ45', 'UniRef50_Q8WZ42')
  ('A0A7K5HKI5', 'UniRef50_Q8WZ42')
  ('A0A7K4TBD4', 'UniRef50_Q8WZ42')
  ('A0A852NBN0', 'UniRef50_Q8WZ42')
  ('A0A7K4YIS6', 'UniRef50_Q8WZ42')
  ('A0A7K7XLB5', 'UniRef50_Q8WZ42')
  ('A0A7K7BSY0', 'UniRef50_Q8WZ42')
  ('A0A7K8HM39', 'UniRef50_Q8WZ42')
  ('A0A7K6WZ68', 'UniRef50_Q8WZ42')
  ('A0A851YGP1', 'UniRef50_Q8WZ42')
  ('A0A7L3R0I3', 'UniRef50_Q8WZ42')
  ('A0A8J4NTA5', 'UniRef50_Q8WZ42')
  ('A0A851LL29', 'UniRef50_Q8WZ42')
  ('A0A852

# Scrap

In [2]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 43.4 MB/s  0:00:00m0:00:0100:01


In [ ]:
import gzip
import xml.etree.ElementTree as ET

def extract_mapping(xml_path, cluster_prefix):
    # xml_path e.g. "uniref90.xml.gz"
    mapping = []
    with gzip.open(xml_path, 'rt') as fh:
        for event, elem in ET.iterparse(fh, events=('end',)):
            if elem.tag == 'entry':
                cluster_id = elem.attrib['id']  # e.g. "UniRef90_P69905"
                for m in elem.findall('.//member'):
                    acc = m.find('dbReference').attrib['id']
                    mapping.append((acc, cluster_id))
                elem.clear()
    return mapping

# Example usage:
map90 = extract_mapping("/scratch/pranamlab/sophie/interactome/interactome/data_files/raw/uniref90/uniref90.xml.gz", "UniRef90_")
map50 = extract_mapping("/scratch/pranamlab/sophie/interactome/interactome/data_files/raw/uniref50/uniref50.xml.gz", "UniRef50_")


In [ ]:
# duckdb_ingest_and_query.py
import duckdb

con = duckdb.connect("uniref.duckdb")

# Adjust paths; DuckDB can read gz/zip directly.
# Expected columns:
#   member_acc, uniref50_id
#   member_acc, uniref90_id
con.execute("""
    CREATE TABLE IF NOT EXISTS uniref50 AS
    SELECT * FROM read_csv_auto('UniRef50_members.tsv', header=True, delim='\t');
""")
con.execute("""
    CREATE TABLE IF NOT EXISTS uniref90 AS
    SELECT * FROM read_csv_auto('UniRef90_members.tsv', header=True, delim='\t');
""")

# Indexes for speed (DuckDB uses zone maps, but this helps equality joins)
con.execute("CREATE INDEX IF NOT EXISTS idx_u50_acc ON uniref50(member_acc);")
con.execute("CREATE INDEX IF NOT EXISTS idx_u90_acc ON uniref90(member_acc);")
con.execute("CREATE INDEX IF NOT EXISTS idx_u50_id  ON uniref50(uniref50_id);")
con.execute("CREATE INDEX IF NOT EXISTS idx_u90_id  ON uniref90(uniref90_id);")

def query_candidates(con, seed_acc: str):
    # 1) Find the seed’s UniRef50 cluster
    con.execute("""
        WITH seed as (
          SELECT u50.uniref50_id
          FROM uniref50 u50
          WHERE u50.member_acc = ?
          LIMIT 1
        ),
        same_u50 AS (
          SELECT u50.member_acc, u50.uniref50_id
          FROM uniref50 u50
          JOIN seed s USING(uniref50_id)
        ),
        same_u50_with_u90 AS (
          SELECT s.member_acc, s.uniref50_id, u90.uniref90_id
          FROM same_u50 s
          LEFT JOIN uniref90 u90 USING(member_acc)
        ),
        seed_u90 AS (
          SELECT u90.uniref90_id FROM uniref90 u90 WHERE u90.member_acc = ? LIMIT 1
        )
        SELECT member_acc
        FROM same_u50_with_u90, seed_u90
        WHERE member_acc <> ?
          AND (uniref90_id IS NULL OR uniref90_id <> seed_u90.uniref90_id);
    """, [seed_acc, seed_acc, seed_acc])
    return [r[0] for r in con.fetchall()]

# Example:
print(query_candidates("Q12778"))


IOException: IO Error: No files found that match the pattern "UniRef50_members.tsv"

LINE 3:     SELECT * FROM read_csv_auto('UniRef50_members.tsv', header=True, delim...
                          ^

In [ ]:
# duckdb_debug_uniref.py
import duckdb
from pathlib import Path

# Point to your processed/debug mapping files
base = Path("/scratch/pranamlab/sophie/interactome/interactome/data_files/processed")
u50_path = base / "uniref50/debug/uniref50_mapping_debug.tsv.gz"
u90_path = base / "uniref90/debug/uniref90_mapping_debug.tsv.gz"

# Use a separate DB for debug so we don't pollute the "real" one
con = duckdb.connect("uniref_debug.duckdb")

# Create tables from gzipped debug TSVs.
# We alias column names to match the original script expectations:
#   uniprot_acc     -> member_acc
#   uniref_cluster  -> uniref50_id / uniref90_id
con.execute(f"""
    CREATE TABLE IF NOT EXISTS uniref50 AS
    SELECT
        uniprot_acc   AS member_acc,
        uniref_cluster AS uniref50_id
    FROM read_csv_auto('{u50_path}', header=True, delim='\t');
""")

con.execute(f"""
    CREATE TABLE IF NOT EXISTS uniref90 AS
    SELECT
        uniprot_acc   AS member_acc,
        uniref_cluster AS uniref90_id
    FROM read_csv_auto('{u90_path}', header=True, delim='\t');
""")

# Optional indexes for faster lookups
con.execute("CREATE INDEX IF NOT EXISTS idx_u50_acc ON uniref50(member_acc);")
con.execute("CREATE INDEX IF NOT EXISTS idx_u90_acc ON uniref90(member_acc);")
con.execute("CREATE INDEX IF NOT EXISTS idx_u50_id  ON uniref50(uniref50_id);")
con.execute("CREATE INDEX IF NOT EXISTS idx_u90_id  ON uniref90(uniref90_id);")

def query_candidates(seed_acc: str):
    con.execute("""
        WITH seed AS (
          SELECT u50.uniref50_id
          FROM uniref50 u50
          WHERE u50.member_acc = ?
          LIMIT 1
        ),
        same_u50 AS (
          SELECT u50.member_acc, u50.uniref50_id
          FROM uniref50 u50
          JOIN seed s USING(uniref50_id)
        ),
        same_u50_with_u90 AS (
          SELECT s.member_acc, s.uniref50_id, u90.uniref90_id
          FROM same_u50 s
          LEFT JOIN uniref90 u90 USING(member_acc)
        ),
        seed_u90 AS (
          SELECT u90.uniref90_id
          FROM uniref90 u90
          WHERE u90.member_acc = ?
          LIMIT 1
        )
        SELECT same_u50_with_u90.member_acc
        FROM same_u50_with_u90, seed_u90
        WHERE same_u50_with_u90.member_acc <> ?
          AND (
                same_u50_with_u90.uniref90_id IS NULL
             OR same_u50_with_u90.uniref90_id <> seed_u90.uniref90_id
          );
    """, [seed_acc, seed_acc, seed_acc])
    return [r[0] for r in con.fetchall()]

test_seed = "A0ABC9HH13"
print(f"Candidates for {test_seed}:")
candidates = query_candidates(test_seed)
print(len(candidates))
print(candidates)


In [9]:
# duckdb_debug_uniref.py
import duckdb
from pathlib import Path

# Point to your processed/debug mapping files
base = Path("/scratch/pranamlab/sophie/interactome/interactome/data_files/processed")
u50_path = base / "uniref50/uniref50_mapping.tsv.gz"
u90_path = base / "uniref90/uniref90_mapping.tsv.gz"

# Use a separate DB for debug so we don't pollute the "real" one
con = duckdb.connect("uniref.duckdb")

# Create tables from gzipped debug TSVs.
# We alias column names to match the original script expectations:
#   uniprot_acc     -> member_acc
#   uniref_cluster  -> uniref50_id / uniref90_id
con.execute(f"""
    CREATE TABLE IF NOT EXISTS uniref50 AS
    SELECT
        uniprot_acc   AS member_acc,
        uniref_cluster AS uniref50_id
    FROM read_csv_auto('{u50_path}', header=True, delim='\t');
""")

con.execute(f"""
    CREATE TABLE IF NOT EXISTS uniref90 AS
    SELECT
        uniprot_acc   AS member_acc,
        uniref_cluster AS uniref90_id
    FROM read_csv_auto('{u90_path}', header=True, delim='\t');
""")

# Optional indexes for faster lookups
con.execute("CREATE INDEX IF NOT EXISTS idx_u50_acc ON uniref50(member_acc);")
con.execute("CREATE INDEX IF NOT EXISTS idx_u90_acc ON uniref90(member_acc);")
con.execute("CREATE INDEX IF NOT EXISTS idx_u50_id  ON uniref50(uniref50_id);")
con.execute("CREATE INDEX IF NOT EXISTS idx_u90_id  ON uniref90(uniref90_id);")

def query_candidates(con, seed_acc: str):
    con.execute("""
        WITH seed AS (
          SELECT u50.uniref50_id
          FROM uniref50 u50
          WHERE u50.member_acc = ?
          LIMIT 1
        ),
        same_u50 AS (
          SELECT u50.member_acc, u50.uniref50_id
          FROM uniref50 u50
          JOIN seed s USING(uniref50_id)
        ),
        same_u50_with_u90 AS (
          SELECT s.member_acc, s.uniref50_id, u90.uniref90_id
          FROM same_u50 s
          LEFT JOIN uniref90 u90 USING(member_acc)
        ),
        seed_u90 AS (
          SELECT u90.uniref90_id
          FROM uniref90 u90
          WHERE u90.member_acc = ?
          LIMIT 1
        )
        SELECT same_u50_with_u90.member_acc
        FROM same_u50_with_u90, seed_u90
        WHERE same_u50_with_u90.member_acc <> ?
          AND (
                same_u50_with_u90.uniref90_id IS NULL
             OR same_u50_with_u90.uniref90_id <> seed_u90.uniref90_id
          );
    """, [seed_acc, seed_acc, seed_acc])
    return [r[0] for r in con.fetchall()]


In [38]:
con = duckdb.connect("uniref.duckdb")
test_seed = "FOXO1_HUMAN"
print(f"Candidates for {test_seed}:")
candidates = query_candidates(con, test_seed)
print(len(candidates))
print(candidates)

Candidates for FOXO1_HUMAN:
2
['UPI00300C9205', 'UPI003004E4AE']


In [39]:
# duckdb_debug_uniref.py
import duckdb
from pathlib import Path

# Point to your processed mapping files
base = Path("/scratch/pranamlab/sophie/interactome/interactome/data_files/processed")
u50_path = base / "uniref50/uniref50_mapping.tsv.gz"
u90_path = base / "uniref90/uniref90_mapping.tsv.gz"

# Use a DB file (full, not debug subset)
con = duckdb.connect("uniref.duckdb")

# (Re)create tables if needed — if you want to force refresh, DROP TABLE first.
con.execute("CREATE TABLE IF NOT EXISTS uniref50 AS "
            f"SELECT uniprot_acc AS member_acc, uniref_cluster AS uniref50_id "
            f"FROM read_csv_auto('{u50_path}', header=True, delim='\t');")

con.execute("CREATE TABLE IF NOT EXISTS uniref90 AS "
            f"SELECT uniprot_acc AS member_acc, uniref_cluster AS uniref90_id "
            f"FROM read_csv_auto('{u90_path}', header=True, delim='\t');")

# Optional indexes for faster lookups
con.execute("CREATE INDEX IF NOT EXISTS idx_u50_acc ON uniref50(member_acc);")
con.execute("CREATE INDEX IF NOT EXISTS idx_u90_acc ON uniref90(member_acc);")
con.execute("CREATE INDEX IF NOT EXISTS idx_u50_id  ON uniref50(uniref50_id);")
con.execute("CREATE INDEX IF NOT EXISTS idx_u90_id  ON uniref90(uniref90_id);")


def query_candidates(con, seed_acc: str):
    con.execute("""
        WITH seed AS (
          SELECT u50.uniref50_id
          FROM uniref50 u50
          WHERE u50.member_acc = ?
          LIMIT 1
        ),
        same_u50 AS (
          SELECT u50.member_acc, u50.uniref50_id
          FROM uniref50 u50
          JOIN seed s USING(uniref50_id)
        ),
        same_u50_with_u90 AS (
          SELECT s.member_acc, s.uniref50_id, u90.uniref90_id
          FROM same_u50 s
          LEFT JOIN uniref90 u90 USING(member_acc)
        ),
        seed_u90 AS (
          SELECT u90.uniref90_id
          FROM uniref90 u90
          WHERE u90.member_acc = ?
          LIMIT 1
        )
        SELECT same_u50_with_u90.member_acc
        FROM same_u50_with_u90, seed_u90
        WHERE same_u50_with_u90.member_acc <> ?
          AND (
                same_u50_with_u90.uniref90_id IS NULL
             OR same_u50_with_u90.uniref90_id <> seed_u90.uniref90_id
          );
    """, [seed_acc, seed_acc, seed_acc])
    return [r[0] for r in con.fetchall()]


def debug_uniref_for_seed(con, seed_acc: str, limit: int = 20):
    print(f"\n================= DEBUG FOR SEED {seed_acc} =================")

    # 0) Show seed rows in uniref50 / uniref90
    print("\n[0] Seed rows in uniref50:")
    rows50 = con.execute(
        "SELECT * FROM uniref50 WHERE member_acc = ?;", [seed_acc]
    ).fetchall()
    print(rows50 if rows50 else "  (none)")

    print("\n[0] Seed rows in uniref90:")
    rows90 = con.execute(
        "SELECT * FROM uniref90 WHERE member_acc = ?;", [seed_acc]
    ).fetchall()
    print(rows90 if rows90 else "  (none)")

    if not rows50:
        print("\n!! Seed is not present in uniref50 at all. Everything else will be empty.")
        return

    # 1) Things in the same UniRef50 cluster
    print("\n[1] Same UniRef50 cluster as seed:")
    same_u50 = con.execute("""
        WITH seed AS (
          SELECT uniref50_id
          FROM uniref50
          WHERE member_acc = ?
          LIMIT 1
        )
        SELECT u50.member_acc, u50.uniref50_id
        FROM uniref50 u50
        JOIN seed s USING(uniref50_id);
    """, [seed_acc]).fetchall()
    print(f"  total: {len(same_u50)}")
    for row in same_u50[:limit]:
        print(" ", row)

    # 2) Things in the same UniRef90 cluster
    if not rows90:
        print("\n[2] Seed has no UniRef90 entry (not in uniref90). Skipping 'same_u90' listing.")
        same_u90 = []
    else:
        print("\n[2] Same UniRef90 cluster as seed:")
        same_u90 = con.execute("""
            WITH seed AS (
              SELECT uniref90_id
              FROM uniref90
              WHERE member_acc = ?
              LIMIT 1
            )
            SELECT u90.member_acc, u90.uniref90_id
            FROM uniref90 u90
            JOIN seed s USING(uniref90_id);
        """, [seed_acc]).fetchall()
        print(f"  total: {len(same_u90)}")
        for row in same_u90[:limit]:
            print(" ", row)

    # 3) Things in the same UniRef50 cluster but NOT in the same UniRef90 cluster
    print("\n[3] Same UniRef50 cluster, but NOT same UniRef90 cluster as seed:")
    diff_u50_not_u90 = con.execute("""
        WITH seed50 AS (
          SELECT uniref50_id
          FROM uniref50
          WHERE member_acc = ?
          LIMIT 1
        ),
        seed90 AS (
          SELECT uniref90_id
          FROM uniref90
          WHERE member_acc = ?
          LIMIT 1
        ),
        same50 AS (
          SELECT u50.member_acc, u50.uniref50_id
          FROM uniref50 u50
          JOIN seed50 s USING(uniref50_id)
        )
        SELECT s.member_acc, s.uniref50_id, u90.uniref90_id
        FROM same50 s
        LEFT JOIN uniref90 u90 USING(member_acc), seed90
        WHERE s.member_acc <> ?
          AND (
               u90.uniref90_id IS NULL
            OR u90.uniref90_id <> seed90.uniref90_id
          );
    """, [seed_acc, seed_acc, seed_acc]).fetchall()

    print(f"  total: {len(diff_u50_not_u90)}")
    for row in diff_u50_not_u90[:limit]:
        print(" ", row)

    print("\n[4] query_candidates() result (should match [3] but only member_acc):")
    candidates = query_candidates(con, seed_acc)
    print(f"  total: {len(candidates)}")
    print(" ", candidates[:limit])


# Example usage:
if __name__ == "__main__":
    test_seed = "UPI00300C9205"
    debug_uniref_for_seed(con, test_seed, limit=30)



================= DEBUG FOR SEED UPI00300C9205 =================

[0] Seed rows in uniref50:
[('UPI00300C9205', 'UniRef50_Q9R1E0')]

[0] Seed rows in uniref90:
  (none)

[1] Same UniRef50 cluster as seed:
  total: 199
  ('FOXO1_PIG', 'UniRef50_Q9R1E0')
  ('FOXO1_HUMAN', 'UniRef50_Q9R1E0')
  ('FOXO1_RAT', 'UniRef50_Q9R1E0')
  ('FOXO1_BOVIN', 'UniRef50_Q9R1E0')
  ('Q8CA93_MOUSE', 'UniRef50_Q9R1E0')
  ('A0A9L0JBQ7_EQUAS', 'UniRef50_Q9R1E0')
  ('A0A0C5GBA6_CAPHI', 'UniRef50_Q9R1E0')
  ('A0A8B8UCW0_CAMFR', 'UniRef50_Q9R1E0')
  ('A0A9W3GQD9_CAMBA', 'UniRef50_Q9R1E0')
  ('A0A452F3R8_CAPHI', 'UniRef50_Q9R1E0')
  ('A0ABM4IHV7_ODOVR', 'UniRef50_Q9R1E0')
  ('A0A4W2GAT4_BOBOX', 'UniRef50_Q9R1E0')
  ('A0ABM4T8A2_BOSIN', 'UniRef50_Q9R1E0')
  ('A0ABN8ZX98_RANTA', 'UniRef50_Q9R1E0')
  ('A0A8C6DYD8_MOSMO', 'UniRef50_Q9R1E0')
  ('A0A6P7EE17_SHEEP', 'UniRef50_Q9R1E0')
  ('A0A452DJI7_BOVIN', 'UniRef50_Q9R1E0')
  ('A0A2Y9N6T9_DELLE', 'UniRef50_Q9R1E0')
  ('A0A452F3S7_CAPHI', 'UniRef50_Q9R1E0')
  ('A0A811Y

In [12]:
from pathlib import Path
import duckdb

base = Path("/scratch/pranamlab/sophie/interactome/interactome/data_files/processed")
u50_path = base / "uniref50/uniref50_mapping.tsv.gz"
u90_path = base / "uniref90/uniref90_mapping.tsv.gz"

con = duckdb.connect(":memory:")  # in-memory, just for this check

print("Looking in raw TSV for Q12778 (UniRef50 mapping):")
rows_file_50 = con.execute(
    f"""
    SELECT *
    FROM read_csv_auto('{u50_path}', header=True, delim='\t')
    WHERE uniprot_acc = 'Q12778';
    """
).fetchall()
print(rows_file_50)

print("\nLooking in raw TSV for Q12778 (UniRef90 mapping):")
rows_file_90 = con.execute(
    f"""
    SELECT *
    FROM read_csv_auto('{u90_path}', header=True, delim='\t')
    WHERE uniprot_acc = 'Q12778';
    """
).fetchall()
print(rows_file_90)


Looking in raw TSV for Q12778 (UniRef50 mapping):
[]

Looking in raw TSV for Q12778 (UniRef90 mapping):
[]


In [23]:
from transformers import AutoTokenizer
tok = AutoTokenizer.from_pretrained("facebook/esm2_t33_650M_UR50D")

/scratch/pranamlab/shared/miniconda3/envs/interact/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
seq = "<mask><mask>C<mask><mask><mask><mask>C<mask><mask>"
tok(seq)

{'input_ids': [0, 32, 32, 23, 32, 32, 32, 32, 23, 32, 32, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [27]:
seq = "xxxCxxx"
seq = seq.upper().replace("X","<mask>")
seq.count("<mask>")

6

In [36]:
import gzip
import xml.etree.ElementTree as ET
from pathlib import Path

def find_uniref_entry(xml_gz_path, query_acc: str, max_print_chars: int = 20000):
    """
    Stream through UniRef XML and print the <entry> block that contains the
    given query accession or ID (e.g. 'Q12778' or 'Q12778_HUMAN').
    """
    xml_gz_path = Path(xml_gz_path)
    print(f"Scanning {xml_gz_path} for {query_acc!r} ...")

    with gzip.open(xml_gz_path, "rt") as fh:
        for event, elem in ET.iterparse(fh, events=("end",)):
            tag = elem.tag.rsplit("}", 1)[-1]  # strip namespace
            if tag != "entry":
                continue

            found = False

            # Look through all dbReference nodes
            for dbref in elem.findall(".//{*}dbReference"):
                db_id = dbref.attrib.get("id", "")
                if query_acc in db_id:
                    found = True
                    break

                # Look at properties (e.g. UniProtKB accession)
                for prop in dbref.findall(".//{*}property"):
                    ptype = prop.attrib.get("type", "")
                    pval = prop.attrib.get("value", "")
                    if query_acc == pval or query_acc in pval:
                        found = True
                        break
                if found:
                    break

            if found:
                print("\n=== FOUND ENTRY ===\n")
                xml_str = ET.tostring(elem, encoding="unicode")
                # Optionally truncate if it's enormous
                if len(xml_str) > max_print_chars:
                    print(xml_str[:max_print_chars])
                    print("\n...[TRUNCATED]...\n")
                else:
                    print(xml_str)
                print("=== END ENTRY ===")
                return

            # free memory
            elem.clear()

    print(f"Did not find any entry containing {query_acc!r}.")


if __name__ == "__main__":
    xml_path = "/scratch/pranamlab/sophie/interactome/interactome/data_files/raw/uniref50/uniref50.xml.gz"

    # Try both accession and ID-style
    find_uniref_entry(xml_path, "Q12778")
    # If that finds nothing, also try:
    # find_uniref_entry(xml_path, "Q12778_HUMAN")


Scanning /scratch/pranamlab/sophie/interactome/interactome/data_files/raw/uniref50/uniref50.xml.gz for 'Q12778' ...

=== FOUND ENTRY ===

<ns0:entry xmlns:ns0="http://uniprot.org/uniref" id="UniRef50_Q9R1E0" updated="2025-10-08">
<ns0:name>Cluster: Forkhead box protein O1</ns0:name>
<ns0:property type="member count" value="200" />
<ns0:property type="common taxon" value="Eutheria" />
<ns0:property type="common taxon ID" value="9347" />
<ns0:property type="GO Molecular Function" value="GO:0003700" />
<ns0:property type="GO Molecular Function" value="GO:0043565" />
<ns0:property type="GO Biological Process" value="GO:0009987" />
<ns0:property type="GO Cellular Component" value="GO:0110165" />
<ns0:representativeMember>
<ns0:dbReference type="UniProtKB ID" id="FOXO1_MOUSE">
<ns0:property type="UniProtKB accession" value="Q9R1E0" />
<ns0:property type="UniProtKB accession" value="Q9JJW4" />
<ns0:property type="UniParc ID" value="UPI0000024130" />
<ns0:property type="UniRef100 ID" value="Un